In [4]:
import os 
from typing import List
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.messages import BaseMessage
from langserve import add_routes
# Load API Keys 
OPENAI_API_KEY=os.environ["OPENAI_API_KEY"]
TAVILY_API_KEY=os.environ["TAVILY_API_KEY"]



# 1. Load Retriever
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings()
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

# 2. Create Tools
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)
search = TavilySearchResults()
tools = [retriever_tool, search]


# 3. Create Agent
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

agent = create_openai_functions_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)



In [6]:
result =agent_executor.invoke({"input": "how can langsmith help with testing? please tell in 2 lines"})
print (result['output'])



> Entering new AgentExecutor chain...
LangSmith can help with testing by providing comprehensive and accurate information about testing methodologies, best practices, and tools, as well as offering guidance and support in implementing effective testing strategies.

> Finished chain.
LangSmith can help with testing by providing comprehensive and accurate information about testing methodologies, best practices, and tools, as well as offering guidance and support in implementing effective testing strategies.


In [7]:
result =agent_executor.invoke({"input": "where to go in DC , answer in 5 line"})
print (result['output'])



> Entering new AgentExecutor chain...
In Washington, D.C., there are several must-visit attractions. Start your day by exploring the iconic National Mall, home to famous landmarks like the Lincoln Memorial and the Washington Monument. Don't miss the opportunity to visit the Smithsonian museums, which offer a wide range of exhibits on art, history, and culture. Take a stroll through the charming Georgetown neighborhood, known for its historic architecture and vibrant shopping and dining scene. For a unique experience, visit the United States Capitol and take a guided tour of the legislative branch of the government. Finally, end your day with a visit to the beautiful and peaceful Tidal Basin, where you can admire the stunning cherry blossoms in the spring.

> Finished chain.
In Washington, D.C., there are several must-visit attractions. Start your day by exploring the iconic National Mall, home to famous landmarks like the Lincoln Memorial and the Washington Monument. Don't miss the o

In [15]:
result =agent_executor.invoke({"input": "where to go in DC"})
print (result['output'])



> Entering new AgentExecutor chain...
There are many places to visit in Washington, D.C. Here are some popular attractions:

1. The National Mall: This is a large park in the center of the city that is home to iconic landmarks such as the Washington Monument, Lincoln Memorial, and the U.S. Capitol.

2. Smithsonian Museums: D.C. is known for its world-class museums, most of which are located on or near the National Mall. Some popular ones include the National Air and Space Museum, National Museum of Natural History, and the National Gallery of Art.

3. The White House: The official residence of the President of the United States. While you can't go inside without a special invitation, you can still visit the exterior and take a guided tour of the surrounding area.

4. The United States Capitol: The home of the U.S. Congress. You can take a guided tour of the Capitol building and learn about the history and workings of the U.S. government.

5. The National Zoo: Located in the Woodley P

In [16]:
print (result)

{'input': 'where to go in DC', 'output': "There are many places to visit in Washington, D.C. Here are some popular attractions:\n\n1. The National Mall: This is a large park in the center of the city that is home to iconic landmarks such as the Washington Monument, Lincoln Memorial, and the U.S. Capitol.\n\n2. Smithsonian Museums: D.C. is known for its world-class museums, most of which are located on or near the National Mall. Some popular ones include the National Air and Space Museum, National Museum of Natural History, and the National Gallery of Art.\n\n3. The White House: The official residence of the President of the United States. While you can't go inside without a special invitation, you can still visit the exterior and take a guided tour of the surrounding area.\n\n4. The United States Capitol: The home of the U.S. Congress. You can take a guided tour of the Capitol building and learn about the history and workings of the U.S. government.\n\n5. The National Zoo: Located in t